In [7]:
import pandas as pd
import numpy as np

In [9]:
vehicles = pd.read_csv("C:\\Users\\joonw\\trav\\vehicles.csv")
policies = pd.read_csv("C:\\Users\\joonw\\trav\\policies.csv")
drivers = pd.read_csv("C:\\Users\\joonw\\trav\\drivers.csv")

# Missing data

In [10]:
def mode_encode(df:pd.DataFrame, col_name:str):
    tmp = df[col_name].mode()
    df[col_name] = df[col_name].fillna(tmp[0])

def mean_encode(df:pd.DataFrame, col_name:str):
    tmp = df[col_name].mean()
    df[col_name] = df[col_name].fillna(tmp)

def median_encode(df:pd.DataFrame, col_name:str):
    tmp = df[col_name].median()
    df[col_name] = df[col_name].fillna(tmp)

In [11]:
policies.isna().any()
drivers.isna().any()
vehicles.isna().any()

Unnamed: 0        False
policy_id         False
car_no            False
ownership_type    False
color              True
age                True
make_model        False
dtype: bool

# Continuous variable: mean? or median? to fill missing values.

If data is skewed, then median may be more appropriate to represent general tendency. 

Below are variables showing non-symmetric patterns so that median be a better measure of central tendency for such datasets.

1. 'quoted_amt: quoted amount' 
2. 'safty_score'

In [12]:
# quoted_amt has a form of "$4,500". Make it float
policies['quoted_amt'] = policies['quoted_amt'].str[1:]
policies['quoted_amt'] = policies['quoted_amt'].astype(str).str.replace(',','').astype(float)


In [13]:
print(f'skewness of quoted amount: {policies['quoted_amt'].skew()}')
print(f'skewness of vehicle age: {vehicles['age'].skew() }')
print(f'skewness of drivers age: {drivers['age'].skew() }')
# Tree model relatively robust to skewed data 0.27 is not practically skewed.
print(f'skewness of drivers safty rating: {drivers['safty_rating'].skew() }')

median_encode(vehicles, 'age')
mode_encode(drivers, 'safty_rating')
mean_encode(policies,'quoted_amt') # no median 

skewness of quoted amount: 3.619782946241044
skewness of vehicle age: 0.45082232812486467
skewness of drivers age: 0.2728148452253024
skewness of drivers safty rating: -0.9619020823577018


In [14]:
mode_encode(policies,'zip')
mean_encode(policies,'credit_score')
mode_encode(policies,'Prior_carrier_grp')
mode_encode(policies,'Cov_package_type')
mode_encode(policies,'CAT_zone')
mode_encode(policies,'convert_ind')
mode_encode(policies,'Agent_cd')
mode_encode(vehicles, 'color')
mode_encode(drivers,'high_education_ind')
mode_encode(drivers,'living_status')

# policies data feature engineering

In [15]:
policies['state_safty'] =  policies['state_id'].apply( lambda x: 1 if x in ['CT','MN' ,'WI'] else (3 if x in ['NY','AL','GA','NJ'] else 2))
policies['discount'] = policies['discount'].apply(lambda x: 1 if x=='Yes' else 0 )
policies['Home_policy_ind'] = policies['Home_policy_ind'].apply(lambda x: 1 if x=='Y' else 0 )
policies['Cov_package_type'] = policies['Cov_package_type'].apply(lambda x: 1 if x=='Low' else (2 if x=='Medium' else 3) )
policies['quote_year'] = pd.to_datetime(policies['Quote_dt']).dt.year
policies['quote_month'] = pd.to_datetime(policies['Quote_dt']).dt.month

policies['primary_parking'] = policies['primary_parking'].apply(lambda x: 1 if x=='street' else (3 if x=='parking garage' or x=='home/driveway' else 2) )

In [16]:
policies = policies.drop(columns=['state_id','Quote_dt','Unnamed: 0','zip','county_name','Agent_cd','Prior_carrier_grp'])
policies

,discount,Home_policy_ind,quoted_amt,credit_score,Cov_package_type,CAT_zone,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,primary_parking,state_safty,quote_year,quote_month
0,1,1,5153.0,613.0,3,2.0,policy_87209,2,1,2,0,3,0.0,Train,3,3,2015,1
1,0,0,9870.0,576.0,1,4.0,policy_89288,1,1,1,2,4,0.0,Test,2,3,2015,2
2,0,0,3090.0,631.0,2,2.0,policy_91413,2,1,2,1,4,0.0,Train,2,3,2018,9
3,0,0,2860.0,573.0,2,3.0,policy_23460,1,2,3,0,5,0.0,Test,2,2,2015,7
4,0,0,14917.0,602.0,2,4.0,policy_71845,2,0,1,2,3,0.0,Train,2,3,2016,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49157,1,1,4042.0,590.0,3,4.0,policy_28284,1,2,2,0,4,0.0,Train,2,2,2018,4
49158,1,1,5743.0,688.0,2,2.0,policy_67016,2,2,2,0,4,0.0,Train,2,3,2017,1
49159,0,0,2230.0,637.0,1,2.0,policy_30163,1,1,3,1,5,0.0,Train,2,2,2018,2
49160,1,1,8428.0,581.0,3,4.0,policy_63982,2,2,2,0,4,1.0,Train,2,3,2016,1


# drivers data feature engineering

When aggregating data over policiyholder, we have to think about whether mean or median could be a better approach. 
For 'safty_score', I filled missings with median but when aggregating this value, I would average over policyholder, groupby().mean(). This is because extreme safty_scores are important and I don't want to suppress this.

However, if I take average on 'driver_age', then I think it might mislead us because this is not about life insurance. The old people may have potential risk in life insurance but this is not true in car insurnace because old people over than 80, they don't get to drive themselves, which means they may has little to do with the potential driving risk of a policy holder. 

However when aggregating driver's age over household, these seniors' age will elevate average age in those household acting like they are potential risk. So I would take square root transformation for 'driver_age' so that I can suppress the influence of extreme values in drivers' age. 

This is beneficial even for tree model by making more balanced splits. 

In [17]:
drivers.rename(columns={'age':'dr_age'}, inplace=True)
drivers=pd.get_dummies(drivers,columns=["gender"])
living_status_dict = {'dependent': 1,'rent': 2,'own': 3}  # scaling less impact on tree model
drivers['living_status']= drivers['living_status'].map(living_status_dict)
drivers['dr_age'] = np.sqrt(drivers['dr_age']  )
drivers1 = drivers.groupby('policy_id')[['dr_age','safty_rating']].mean()
# for car insurance, it does not make sense to aggreate driver's age over each household or policy_id because 
# when I looked at the age, there were significant number of people with age over 80 90 or even 100, and I don't think they will drive 
# by themselves, if this was a life insurance it is matter but this is a car insurance 
# I may transform age data by square root transformation and this will compress the influence of older ages
#  linear model is not gonna work well with age because the relation 
drivers = drivers.drop(columns=['Unnamed: 0'])
drivers = drivers.groupby('policy_id').mean()

# vehicles data feature engineering

In [18]:
vehicles['col_safty'] =  vehicles['color'].apply( lambda x: 1 if x in ['gray','white' ,'silver'] else (3 if x in ['blue','red','black'] else 2))
vehicles['model_risk'] = vehicles['make_model'].str.contains('BMW|DODGE|HYUNDAI|NISSAN|INFINITY') 
owner_dict = {'leased': 1,'loaned': 2,'owned': 3}  # scaling less impact on tree model
vehicles['ownership_type']= vehicles['ownership_type'].map(owner_dict)
vehicles = vehicles.drop(columns=['Unnamed: 0','color' ,'make_model'])
vehicles= vehicles.groupby('policy_id').mean()
vehicles.rename(columns={'age':'car_age'}, inplace=True)

# policies.apply(lambda x: x.unique(), axis=0)
# drivers.apply(lambda x: x.unique(), axis=0)
# vehicles.apply(lambda x: x.unique(), axis=0)

# merge data sets

In [19]:
df1 = pd.merge(policies, vehicles, on = 'policy_id', how='inner')
df = pd.merge(df1, drivers, on = 'policy_id', how='inner')
df = df.drop(columns=['policy_id'])
df.to_csv("C:\\Users\\joonw\\trav\\trav_dataset1.csv", index=False)
df

,discount,Home_policy_ind,quoted_amt,credit_score,Cov_package_type,CAT_zone,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,...,ownership_type,car_age,col_safty,model_risk,living_status,dr_age,safty_rating,high_education_ind,gender_F,gender_M
0,1,1,5153.0,613.0,3,2.0,2,1,2,0,...,2.50,5.25,1.25,0.50,2.000000,7.009449,74.0,0.5,0.500000,0.500000
1,0,0,9870.0,576.0,1,4.0,1,1,1,2,...,2.25,2.75,2.50,0.25,2.000000,7.483315,62.0,1.0,0.000000,1.000000
2,0,0,3090.0,631.0,2,2.0,2,1,2,1,...,2.50,7.00,1.50,0.25,2.000000,6.986819,70.0,0.5,0.500000,0.500000
3,0,0,2860.0,573.0,2,3.0,1,2,3,0,...,2.50,5.50,2.00,0.50,2.000000,5.477226,90.0,1.0,1.000000,0.000000
4,0,0,14917.0,602.0,2,4.0,2,0,1,2,...,2.75,4.50,2.25,0.50,3.000000,7.423662,81.5,1.0,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49157,1,1,4042.0,590.0,3,4.0,1,2,2,0,...,2.50,5.00,2.25,0.00,2.000000,7.211103,90.0,1.0,0.000000,1.000000
49158,1,1,5743.0,688.0,2,2.0,2,2,2,0,...,2.25,6.25,1.25,0.25,2.000000,8.234404,73.5,0.0,0.500000,0.500000
49159,0,0,2230.0,637.0,1,2.0,1,1,3,1,...,3.00,7.00,1.00,0.50,1.000000,4.358899,39.0,0.0,1.000000,0.000000
49160,1,1,8428.0,581.0,3,4.0,2,2,2,0,...,1.80,4.80,1.60,0.60,2.000000,7.264148,78.0,0.5,1.000000,0.000000
